# Airbyte Gong (已弃用)

注意：此连接器特定的加载器已被弃用。请改用[`AirbyteLoader`](/docs/integrations/document_loaders/airbyte)。

>[Airbyte](https://github.com/airbytehq/airbyte)是一个用于从API、数据库和文件到数据仓库和数据湖的ELT管道的数据集成平台。它拥有最大的ELT连接器目录，可连接到数据仓库和数据库。

此加载器将Gong连接器公开为文档加载器，允许您将各种Gong对象作为文档加载。

## 安装

首先，您需要安装`airbyte-source-gong` Python包。

In [ ]:
%pip install --upgrade --quiet  airbyte-source-gong

## 示例

查看[Airbyte文档页面](https://docs.airbyte.com/integrations/sources/gong/)了解有关如何配置读取器的详细信息。
配置对象应遵循的JSON模式可以在Github上找到：[https://github.com/airbytehq/airbyte/blob/master/airbyte-integrations/connectors/source-gong/source_gong/spec.yaml](https://github.com/airbytehq/airbyte/blob/master/airbyte-integrations/connectors/source-gong/source_gong/spec.yaml)。

一般形状如下所示：
```python
{
  "access_key": "<访问密钥名称>",
  "access_key_secret": "<访问密钥秘钥>",
  "start_date": "<开始检索记录的日期，ISO格式，例如 2020-10-20T00:00:00Z>",
}
```

默认情况下，所有字段都作为元数据存储在文档中，而文本设置为空字符串。通过转换读取器返回的文档来构建文档的文本。

In [ ]:
from langchain_community.document_loaders.airbyte import AirbyteGongLoader

config = {
    # 您的gong配置
}

loader = AirbyteGongLoader(
    config=config, stream_name="calls"
)  # 查看上面链接的文档以获取所有流的列表

现在您可以按照常规方式加载文档

In [ ]:
docs = loader.load()

由于`load`返回一个列表，它会阻塞直到所有文档加载完成。要更好地控制这个过程，您也可以使用`lazy_load`方法，它返回一个迭代器：

In [ ]:
docs_iterator = loader.lazy_load()

请记住，默认情况下页面内容为空，元数据对象包含记录中的所有信息。要处理文档，请创建一个继承基础加载器的类，并自己实现`_handle_records`方法：

In [ ]:
from langchain_core.documents import Document


def handle_record(record, id):
    return Document(page_content=record.data["title"], metadata=record.data)


loader = AirbyteGongLoader(
    config=config, record_handler=handle_record, stream_name="calls"
)
docs = loader.load()

## 增量加载

一些流允许增量加载，这意味着源会跟踪已同步的记录，并且不会再次加载它们。这对于数据量大且频繁更新的源很有用。

要利用这一点，存储加载器的`last_state`属性，并在再次创建加载器时传入它。这将确保只加载新记录。

In [ ]:
last_state = loader.last_state  # 安全存储

incremental_loader = AirbyteGongLoader(
    config=config, stream_name="calls", state=last_state
)

new_docs = incremental_loader.load()